In [1]:
import pandas as pd
import pathlib
import os

In [3]:
# Build path to data directory using pathlib
current_dir = pathlib.Path(os.getcwd())
data_directory = current_dir.parents[1] / "data"

# Clean price index dataframe
prices_index_filename = data_directory / "price_index_mexico.csv"
prices_index_df = pd.read_csv(prices_index_filename)
prices_index_df = prices_index_df[["CONCEPTO", "VALOR", "FECHA"]]
prices_index_df["CONCEPTO"] = prices_index_df["CONCEPTO"].astype(str)

# Keep only variable of interest (most widely used price index in Mexico)
prices_index_df = prices_index_df[prices_index_df["CONCEPTO"].str.contains("INPC", na=False)]
prices_index_df["CONCEPTO"] = "INPC"
prices_index_df["FECHA"] = pd.to_datetime(prices_index_df["FECHA"], dayfirst=True)

# Keep only data from 2012 to present
date_cutoff = pd.Timestamp("2012-01-01")
prices_index_df = prices_index_df[prices_index_df["FECHA"] >= date_cutoff]
prices_index_df["Deflactor"] = 100 / prices_index_df["VALOR"]
prices_index_df.columns = ["CONCEPTO", "Index", "Date", "Deflactor"]
prices_index_df = prices_index_df[["Date", "Deflactor"]]
prices_index_df.head()




,Date,Deflactor
759,2012-01-01,1.276437
766,2012-02-01,1.273848
773,2012-03-01,1.273117
780,2012-04-01,1.277123
787,2012-05-01,1.281167


In [5]:
# Keep only observations of interest from spending dataset

public_spenditure_filename = data_directory / "public_spenditure_mexico.xlsx"
public_spenditure_df = pd.read_excel(public_spenditure_filename, sheet_name=None, skiprows=17)
public_spenditure_df = public_spenditure_df["Hoja1"]
public_spenditure_df.columns = ["Date", "Spenditure"]
public_spenditure_df["Date"] = pd.to_datetime(public_spenditure_df["Date"])

# Use the same date_cutoff as in prices index
public_spenditure_df = public_spenditure_df[public_spenditure_df["Date"] >= date_cutoff]
public_spenditure_df.head()


/Users/danielm/altair_env/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Date,Spenditure
420,2012-01-01,243185.7
421,2012-02-01,479040.1
422,2012-03-01,709299.9
423,2012-04-01,922097.4
424,2012-05-01,1121847.7


In [11]:
# Merge data

public_spenditure_df = public_spenditure_df.merge(prices_index_df, on="Date", how="left")
public_spenditure_df.head()

,Date,Spenditure,Deflactor
0,2012-01-01,243185.7,1.276437
1,2012-02-01,479040.1,1.273848
2,2012-03-01,709299.9,1.273117
3,2012-04-01,922097.4,1.277123
4,2012-05-01,1121847.7,1.281167


In [12]:
# Calculate the real spending using the prices index

public_spenditure_df["Spenditure_real"] = public_spenditure_df["Spenditure"] * public_spenditure_df["Deflactor"]
public_spenditure_df.head()

,Date,Spenditure,Deflactor,Spenditure_real
0,2012-01-01,243185.7,1.276437,3.104113e+05
1,2012-02-01,479040.1,1.273848,6.102242e+05
2,2012-03-01,709299.9,1.273117,9.030216e+05
3,2012-04-01,922097.4,1.277123,1.177632e+06
4,2012-05-01,1121847.7,1.281167,1.437275e+06


In [13]:
# Upload clean dataset
filename = data_directory / "spenditure_clean.csv"
public_spenditure_df.to_csv(filename)

In [6]:
# Clean debt dataset
# Data is in thousands of millions pesos

debt_filename = data_directory / "Consulta_20241102-152135979.csv"
debt_df = pd.read_csv(debt_filename)
debt_df.columns = ["Date", "Debt"]
debt_df["Date"] = pd.to_datetime(debt_df["Date"], format="%d/%m/%y")
debt_df = debt_df.merge(prices_index_df, on="Date", how="left")

# Calculate real debt using prices index
debt_df["Debt_real"] = debt_df["Debt"] * debt_df["Deflactor"]

clean_filename = data_directory / "debt_clean.csv"
debt_df.to_csv(clean_filename)
debt_df.head(13)


,Date,Debt,Deflactor,Debt_real
0,2012-01-01,4459.58,1.276437,5692.374794
1,2012-02-01,4411.78,1.273848,5619.936259
2,2012-03-01,4417.53,1.273117,5624.031651
3,2012-04-01,4489.12,1.277123,5733.159433
4,2012-05-01,4630.68,1.281167,5932.675735
5,2012-06-01,4713.62,1.275288,6011.222532
6,2012-07-01,4827.28,1.268168,6121.802669
7,2012-08-01,4799.95,1.264374,6068.930598
8,2012-09-01,4850.03,1.258826,6105.342135
9,2012-10-01,4810.31,1.252489,6024.859187
